## Dropping Columns, Removing Null Values, and Filtering Target Values  

In [1]:
import pandas as pd 
import numpy as np 


df = pd.read_csv(r'https://media.githubusercontent.com/media/norrisja/DS3000-Project/master/hmda_2017_ma_all-records_labels.csv')
dropped_columns = ['as_of_year', 'respondent_id', 'agency_name', 'agency_abbr', 'agency_code',
                  'loan_type', 'property_type', 'loan_purpose', 'owner_occupancy', 'preapproval_name',
                  'preapproval', 'action_taken_name', 'msamd_name', 'msamd', 'state_name',
                  'state_abbr', 'state_code', 'county_code', 'census_tract_number', 'applicant_ethnicity',
                  'co_applicant_ethnicity', 'applicant_race_1', 'applicant_race_name_2', 'applicant_race_2',
                   'applicant_race_name_3', 'applicant_race_3', 'applicant_race_name_4', 'applicant_race_4',
                  'applicant_race_name_5', 'applicant_race_5', 'co_applicant_race_1', 'co_applicant_race_name_2',
                  'co_applicant_race_2', 'co_applicant_race_name_3', 'co_applicant_race_3', 'co_applicant_race_name_4',
                  'co_applicant_race_4', 'co_applicant_race_name_5', 'co_applicant_race_5', 'applicant_sex',
                  'co_applicant_sex', 'purchaser_type', 'denial_reason_name_1', 'denial_reason_1', 'denial_reason_name_2',
                  'denial_reason_2', 'denial_reason_name_3', 'denial_reason_3', 'rate_spread', 'hoepa_status',
                  'lien_status', 'edit_status_name', 'edit_status', 'sequence_number', 'application_date_indicator',
                  'purchaser_type_name', 'co_applicant_sex_name', 'co_applicant_race_name_1' ] 


df = df.drop(columns=dropped_columns)
df = df.replace('Information not provided by applicant in mail, Internet, or telephone application', np.NaN)
df = df.dropna()
df = df.loc[(df['action_taken'] == 1) | (df['action_taken'] == 3)]

features_count = df['action_taken'].value_counts()
features_count
approved = features_count[1] 
denied = features_count[3] 
percent_approved = round((approved / (denied + approved)) * 100, 2)
percent_denied = round((denied / (denied + approved)) * 100, 2)
print(f'Percent Approved: {percent_approved}%\nPercent Denied:   {percent_denied}%')

C:\Users\peter\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (34,36,38,44,46,59,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Percent Approved: 84.44%
Percent Denied:   15.56%


## Feature Engineering Using One-Hot Encoding 

In [2]:
from sklearn.preprocessing import OneHotEncoder


features = df.drop(columns=['action_taken'])

ohe_cols = ['loan_type_name', 'property_type_name', 'loan_purpose_name', 'owner_occupancy_name',
           'county_name','applicant_ethnicity_name', 'co_applicant_ethnicity_name', 'applicant_race_name_1', 
           'applicant_sex_name', 'hoepa_status_name', 'lien_status_name']

ohe_features = features[ohe_cols]
not_ohe_features_df = features.drop(columns=ohe_features)

encoder = OneHotEncoder(sparse = False)
encoded_df = encoder.fit_transform(ohe_features)
encoder.get_feature_names()
features_df = pd.DataFrame(encoded_df, columns = encoder.get_feature_names())
ohe_features = pd.concat([features_df.reset_index(drop=True), not_ohe_features_df.reset_index(drop=True)], axis=1)
target = df['action_taken']

## Feature Engineering Using SVM Iterative Feature Selection

In [3]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(ohe_features, target, random_state=3000)

select = RFE(DecisionTreeClassifier(), n_features_to_select = 10)
select.fit(X_train, y_train)

X_train_selected = select.transform(X_train)
X_test_selected = select.transform(X_test)

model = DecisionTreeClassifier().fit(X=X_train, y=y_train)

print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

model = DecisionTreeClassifier().fit(X=X_train_selected, y=y_train)
print(model.score(X_train_selected, y_train))
print(model.score(X_test_selected, y_test))

0.9986770034549591
0.7676011831028642
0.998264093705233
0.7640114265490305


## Creating the Final DataFrame and Sending it to a CSV.

In [6]:
mask = pd.Series(select.get_support())
cols = ohe_features.columns
filtered_cols = cols[mask]
filtered_features = ohe_features[filtered_cols]
final_df = pd.concat([target.reset_index(drop=True), filtered_features.reset_index(drop=True)], axis=1)
final_df.to_csv(r'https://media.githubusercontent.com/media/norrisja/DS3000-Project/master/final_data.csv', index=False)


,action_taken,x2_Home purchase,x8_Male,loan_amount_000s,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units
0,1,1.0,1.0,550,179.0,4755.0,27.440001,104800.0,133.389999,1352.0,1949.0
1,3,0.0,1.0,288,75.0,6609.0,3.190000,94300.0,128.529999,2038.0,2474.0
2,3,0.0,0.0,200,66.0,6139.0,11.760000,94300.0,146.729996,1793.0,2039.0
3,3,0.0,0.0,243,57.0,5232.0,61.240002,94300.0,56.450001,1087.0,1047.0
4,1,0.0,1.0,420,199.0,5521.0,2.230000,94300.0,152.429993,1721.0,2128.0
...,...,...,...,...,...,...,...,...,...,...,...
158222,3,1.0,1.0,60,78.0,2076.0,2.940000,69000.0,107.160004,711.0,979.0
158223,3,1.0,1.0,35,28.0,2076.0,2.940000,69000.0,107.160004,711.0,979.0
158224,3,0.0,0.0,10,87.0,7314.0,41.110001,84000.0,89.230003,1316.0,2470.0
158225,3,0.0,1.0,5,29.0,1913.0,24.150000,66600.0,52.750000,179.0,664.0


## Read the CSV from GitHub

In [8]:
final_df = pd.read_csv(r'https://media.githubusercontent.com/media/norrisja/DS3000-Project/master/final_data.csv')
final_df

,action_taken,x2_Home purchase,x8_Male,loan_amount_000s,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units
0,1,1.0,1.0,550,179.0,4755.0,27.440001,104800.0,133.389999,1352.0,1949.0
1,3,0.0,1.0,288,75.0,6609.0,3.190000,94300.0,128.529999,2038.0,2474.0
2,3,0.0,0.0,200,66.0,6139.0,11.760000,94300.0,146.729996,1793.0,2039.0
3,3,0.0,0.0,243,57.0,5232.0,61.240002,94300.0,56.450001,1087.0,1047.0
4,1,0.0,1.0,420,199.0,5521.0,2.230000,94300.0,152.429993,1721.0,2128.0
...,...,...,...,...,...,...,...,...,...,...,...
158222,3,1.0,1.0,60,78.0,2076.0,2.940000,69000.0,107.160004,711.0,979.0
158223,3,1.0,1.0,35,28.0,2076.0,2.940000,69000.0,107.160004,711.0,979.0
158224,3,0.0,0.0,10,87.0,7314.0,41.110001,84000.0,89.230003,1316.0,2470.0
158225,3,0.0,1.0,5,29.0,1913.0,24.150000,66600.0,52.750000,179.0,664.0
